
### Installing Required Packages for Hugging Face and LangChain
In this block, we install the necessary packages for using **Hugging Face Hub** and **LangChain**, which include community modules and tools for building language model applications.

In [ ]:
# Install required packages for Hugging Face and LangChain usage

print("Installing packages... this can take a minute or two.")

%pip install -q langchain langchain-community langchain-huggingface langchain-openai chromadb google-search-results

print("All required packaged installed and ready!")

### Setting Up Hugging Face Access Token
We configure our environment with **access token** for Hugging Face, OpenAI and Google Search. This is necessary for programmatic access to models and datasets available on Hugging Face Hub, as well as access to OpenAI, and Google Search.

In [ ]:
# Import os library for environment variable manipulation
import os

# Note: Ensure these are securely managed and not hard-coded in production.

# Set your Hugging Face Hub Access token (replace 'your_token_here' with your actual token)
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "paste your huggingface access token here"

# Set OpenAI key for accessing GPT-3 and GPT-3 models (paid account)
os.environ["OPENAI_API_KEY"] = "paste your openai access key here"

# SERPAPI API key for accessing Google Search API (Free account is limited to 100 API calls per month)
os.environ["SERPAPI_API_KEY"] = "paste your google serpapi key here"

print("All Set!")

### Building a Simple Q&A Chatbot Using LangChain
We will set up a basic **Q&A chatbot** using **LangChain** and a **small language model** from Hugging Face. This demonstrates chaining models and using templates.
Exercises:
- Experiment with different **small** models (uncomment LANGUAGE_MODEL to test alternatives).
- Adjust the temperature setting (TEMP of 0.9 for the most varied responses, 0.1 for the least varied).
- Try using different substitution variables (e.g., 'language':, set to "Spanish").
- Now try **OpenAI's GPT** large language model (by uncomment corresponding line below)
- Last, Alter Prompt to trigger rude response (e.g, ... you dummy)

In [ ]:
# Import necessary libraries for building a LangChain chatbot
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint

# Initialize a small language model from Hugging Face
#LANGUAGE_MODEL = "meta-llama/Llama-2-7b"
#LANGUAGE_MODEL = "google/flan-t5-small"
#LANGUAGE_MODEL = "bigscience/bloom" # Play with temperature to get different results
#LANGUAGE_MODEL = "tiiuae/falcon-7b-instruct" # Works okay, best with temperature: 0.1-0.5
LANGUAGE_MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1" # Works okay with lower temperatures

# Increase to cause vary reponses (.9 max, 0 min)
TEMP = .2

# Define LM to one fo the (smaller) Hugging Face models
llm = HuggingFaceEndpoint( repo_id=LANGUAGE_MODEL, temperature=TEMP)

# Uncomment to define OpenAI GPT model (A large language model)
#llm = ChatOpenAI(temperature=.1)  # uncomment to use OpenAI GPT model (A large language model)

# Define a prompt template for the chatbot
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Please respond in {language} in 20 words or less. {validate}'),
    ('human', '{input}')
])

# Define the chatbot chain where the prompt is sent to the language model
chain = prompt | llm

# Invoke the chatbot with a sample input
response = chain.invoke({'input': 'Who is the tallest superhero?',
                         'language': 'English',
                         'validate': 'Keep it clean'})

# Print the chatbot's response
print(response)

### RAG-Based Document Summarization
Demonstrates a **Retrieval-Augmented Generation** (RAG) process by splitting a document into chunks, embedding it into a searchable database, retrieving relevant information, and generating a summary using a language model.
Exercises:
- Change query_text, perhaps something to do with "radon gas".
- Change Language to Spanish in prompt template
- Update system prompt to include special formatting (E.g., HTML, JSON)

In [ ]:
# Import Embeddings model for RAG, and Chroma in memory vector database
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.vectorstores.chroma import Chroma
import requests
import logging

logging.getLogger("langchain_text_splitters.base").setLevel(logging.ERROR)

# Load the document from a GITHUB, normalizing special characters
DOC_URL = "https://jerrycuomo.github.io/Think_Artificial_Intelligence/datasets/EPA-consumer-safety-safe-water.txt"
full_text = requests.get(DOC_URL).text.replace("\r\n", "\n").replace("\r", "\n")

# Chunk the document and tokenize
text_splitter = CharacterTextSplitter(chunk_size=300)
texts = text_splitter.split_text(full_text)
print(f"Document has been split into {len(texts)} chunks")

# Initialize the embedding model and create a searchable database from the chunked texts
embeddings = OpenAIEmbeddings()
db = Chroma.from_texts(texts, embeddings)

# Retrieving the context from the DB using similarity search
# query_text = "Can Radon gas enter your home?"
query_text = "What is considered safe drinking water?"
results = db.similarity_search(query_text, 1)

# Configure the prompt template for concise summarization
prompt = ChatPromptTemplate.from_messages([
    ("system", "Please summarize in {language} in 25 words or less. {validate}"),
    ("human", "{question} {input}")
])

# Set up the LangChain LLM for processing the information retrieved, defining the sequence for action
llm = ChatOpenAI(temperature=.2)
chain = prompt | llm

# Execute the chain on the first retrieved document, specifying the output language and summary style
response = chain.invoke({"question": query_text,
                         "input": results[0].page_content,
                         "language": "English",
                         "validate": "Ensure clarity and accessibility"})
print(response.content)

### Langchain Agent (Skilled in Web Search and Math)
This program initializes an Langchain-based agent equipped with search and math tools, allowing it to answer complex queries by retrieving information from the web and performing calculations dynamically.

Exercise:
- Try difference queries by uncommenting options below

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

# Initialize the OpenAI agent with a specific temperature setting
llm = ChatOpenAI(temperature=.2)

# Load necessary tools for the agent, including SERPAPI for searches and llm-math for mathematical queries
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Initialize the agent with the loaded tools, setting it to a zero-shot react description mode for dynamic response handling
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Define a query and invoke the agent to handle it, demonstrating the agent's capability to generate and evaluate responses
#query = "How much would it cost to fill a pool the size of an Olympic swimming pool using the average water price in Los Angeles?"
#query = "What’s the average monthly salary in Switzerland, and how long would it take a person earning that salary to save enough to buy a Tesla Model S, factoring in living costs of 70% of their income?"
#query = "What's the current price of Tesla stock, and how much would 15 shares cost?"

query = "What was the total score of the Super Bowl in the year Justin Bieber was born?"

agent.invoke(query)
